In [ ]:

%run ../../load_magic/storage.py
%pprint
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%who

In [ ]:

def plot_colortable(colors_dict, title, sort_colors=True, emptycols=0):

    cell_width = 212
    cell_height = 22
    swatch_width = 48
    margin = 12
    topmargin = 40

    # Sort colors_dict by hue, saturation, value and name.
    if sort_colors is True:
        by_hsv = sorted((tuple(mcolors.rgb_to_hsv(mcolors.to_rgb(color))),
                         name)
                        for name, color in colors_dict.items())
        names = [name for hsv, name in by_hsv]
    else:
        names = list(colors_dict)

    n = len(names)
    ncols = 4 - emptycols
    nrows = n // ncols + int(n % ncols > 0)

    width = cell_width * 4 + 2 * margin
    height = cell_height * nrows + margin + topmargin
    dpi = 72

    fig, ax = plt.subplots(figsize=(width / dpi, height / dpi), dpi=dpi)
    fig.subplots_adjust(margin/width, margin/height,
                        (width-margin)/width, (height-topmargin)/height)
    ax.set_xlim(0, cell_width * 4)
    ax.set_ylim(cell_height * (nrows-0.5), -cell_height/2.)
    ax.yaxis.set_visible(False)
    ax.xaxis.set_visible(False)
    ax.set_axis_off()
    ax.set_title(title, fontsize=24, loc="left", pad=10)

    for i, name in enumerate(names):
        row = i % nrows
        col = i // nrows
        y = row * cell_height

        swatch_start_x = cell_width * col
        swatch_end_x = cell_width * col + swatch_width
        text_pos_x = cell_width * col + swatch_width + 7

        ax.text(text_pos_x, y, name, fontsize=14,
                horizontalalignment='left',
                verticalalignment='center')

        ax.hlines(y, swatch_start_x, swatch_end_x,
                  color=colors_dict[name], linewidth=18)

In [ ]:

plot_colortable(mcolors.BASE_COLORS, "Base Colors",
                sort_colors=True, emptycols=1)

In [ ]:

colors_dict = mcolors.BASE_COLORS
colors_dict

In [ ]:

# Red, Green, Blue
{name: mcolors.to_rgb(color) for name, color in colors_dict.items()}

In [ ]:

# Hue, Saturation, Value
{name: tuple(mcolors.rgb_to_hsv(mcolors.to_rgb(color))) for name, color in colors_dict.items()}

In [ ]:

columns_list = ['Red', 'Green', 'Blue']
def colors_dict_to_df(colors_dict):
    rows_list = []
    index_list = []
    for base_name, color_tuple in colors_dict.items():
        row_dict = {}
        index_list.append(base_name)
        for i, color_value in enumerate(columns_list):
            row_dict[color_value] = color_tuple[i]
        rows_list.append(row_dict)
    df = pd.DataFrame(rows_list, columns=columns_list, index=index_list)
    
    return df

In [ ]:

base_colors_df

In [ ]:

from sklearn.decomposition import PCA

pca = PCA(n_components = 2)
X2D = pca.fit_transform(X)
fig = plt.figure()
ax = fig.add_subplot(111, aspect='equal')

ax.plot(X2D[:, 0], X2D[:, 1], 'k+')
ax.plot(X2D[:, 0], X2D[:, 1], 'k.')
ax.plot([0], [0], 'ko')
ax.arrow(0, 0, 0, 1, head_width=0.05, length_includes_head=True, head_length=0.1, fc='k', ec='k')
ax.arrow(0, 0, 1, 0, head_width=0.05, length_includes_head=True, head_length=0.1, fc='k', ec='k')
ax.set_xlabel('$z_1$', fontsize=18)
ax.set_ylabel('$z_2$', fontsize=18, rotation=0)
ax.axis([-1.5, 1.3, -1.2, 1.2])
ax.grid(True)
save_fig('dataset_2d_plot')

In [ ]:

plot_colortable(mcolors.TABLEAU_COLORS, "Tableau Palette",
                sort_colors=True, emptycols=2)

In [ ]:

#sphinx_gallery_thumbnail_number = 3
plot_colortable(mcolors.CSS4_COLORS, "CSS Colors")

In [ ]:

for name, color in mcolors.XKCD_COLORS.items():
    print(name, color)
    print(mcolors.to_rgb(color))
    break

In [ ]:

import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN

colors_dict = {name: mcolors.to_rgb(color) for name, color in mcolors.XKCD_COLORS.items()}
xkcd_colors_df = colors_dict_to_df(colors_dict)
X = xkcd_colors_df.values
pca_ndarray = PCA(n_components=2).fit_transform(X)

In [ ]:

eps = 0.09
min_samples = 1
db = DBSCAN(eps=eps, min_samples=min_samples).fit(pca_ndarray)
labels_ndarray = db.labels_
unique_labels = np.unique(labels_ndarray)
unique_labels_count = len(unique_labels)
if (unique_labels_count > 4) and (unique_labels_count < 8):
    print('np.unique(DBSCAN(eps={}, min_samples={}).fit(pca_ndarray).labels_) = {}'.format(eps, min_samples, unique_labels))

In [ ]:

kwargs = dict(textcoords='offset points', ha='left', va='bottom',
              bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
              arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
for col, label, x, y in zip(labels_ndarray, merge_df.index,
                            pca_ndarray[:, 0], pca_ndarray[:, 1]):
    if (label == 'Franks'):
        annotation = plt.annotate('{} (and everyone else)'.format(label), xy=(x, y),
                                  xytext=(20, 10), **kwargs)
    elif (col == 1):
        annotation = plt.annotate(label, xy=(x, y), xytext=(10, -20), **kwargs)
    elif (col == 2):
        annotation = plt.annotate(label, xy=(x, y), xytext=(-10, 20), **kwargs)
    elif (col == 3):
        annotation = plt.annotate(label, xy=(x, y), xytext=(-20, 20), **kwargs)
    elif (col == 4):
        annotation = plt.annotate(label, xy=(x, y), xytext=(-10, 20), **kwargs)

In [ ]:

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, autoscale_on=True)
cmap = plt.get_cmap('viridis_r')
path_collection = ax.scatter(pca_ndarray[:, 0], pca_ndarray[:, 1],
                             c=labels_ndarray, edgecolors=(0, 0, 0), cmap=cmap)
title_text = 'Dimension Reduced Scatterplot of the XKCD Colors'
text = plt.title(title_text)
xticklabel_obj = plt.xticks([])
yticklabel_obj = plt.yticks([])

In [ ]:

# Optionally plot the XKCD colors (Caution: will produce large figure)
file_name = 'XKCD_Colors.png'
png_folder = os.path.join(SAVES_FOLDER, 'png')
file_path = os.path.join(png_folder, file_name)
xkcd_fig = plot_colortable(mcolors.XKCD_COLORS, "XKCD Colors")
xkcd_fig.savefig(file_path)

In [ ]:

patriline_df = load_object('patriline_df')
min_year = patriline_df['Year of Birth'].min()
generations_df = load_object('generations_df')
match_series = (generations_df['birth_year_begin'] >= min_year)
generations_df[match_series]['saeculum_name'].unique()

In [ ]:

unique_color_groups_count = len(generations_df[match_series]['saeculum_name'].unique())
print(type(mcolors.XKCD_COLORS))
dir(mcolors.XKCD_COLORS)